In [ ]:
!pip -q install langchain openai faiss-cpu sentence-transformers pypdf tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

PAPER_DIR   = "/content/drive/MyDrive/ProfS/ProfS_Papers"
INDEX_PATH  = "/content/drive/MyDrive/ProfS/ProfS_FAISS"

Mounted at /content/drive


In [ ]:
!pip install -Uq langchain-community pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00


In [ ]:
# 1 · load + chunk papers
import glob, os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

pdf_files = glob.glob(os.path.join(PAPER_DIR, "**/*.pdf"), recursive=True)
docs = []
for pdf in pdf_files:
    docs += PyPDFLoader(pdf).load()

print(f"Loaded {len(pdf_files)} PDFs  →  {len(docs)} pages")

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
chunks   = splitter.split_documents(docs)
print(f"Split into {len(chunks):,} chunks")

Loaded 5 PDFs  →  88 pages
Split into 495 chunks


In [ ]:
# 2 · embed with small model + build / load FAISS index
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from pathlib import Path

emb = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# tag every PDF chunk so we can recognise genuine sources
for c in chunks:
    c.metadata["type"] = "paper"

# (re)build
vectordb = FAISS.from_documents(chunks, emb)
vectordb.save_local(INDEX_PATH)
print("FAISS index rebuilt with latest PDFs ✔️")

# (lower → stricter)
retriever = vectordb.as_retriever(
    search_kwargs={"k": 4, "score_threshold": 0.75}
)

<ipython-input-5-f88a56756d86>:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  emb = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

FAISS index rebuilt with latest PDFs ✔️


In [ ]:
# 3
from langchain.prompts.chat import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

# Define system prompt
SYSTEM_PROMPT = (
    "You are Professor S, a kind, patient, slightly quirky CS professor. "
    "You often say 'emm…', 'actually', 'fascinating', 'oh I see', and begin with "
    "'Fantastic question!'. End every answer with ':D '. "
    "Use the following context to inform your answer. If the context is relevant, prioritize it. "
    "If the question is unrelated to the context, use your general knowledge to provide a helpful answer."
    "\n\nContext:\n{context}"
)


# Prompt structure using system + memory + question
prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(SYSTEM_PROMPT),
    MessagesPlaceholder(variable_name="chat_history"),
    HumanMessagePromptTemplate.from_template("{question}")
])

CONDENSE_Q_SYSTEM = (
    "You are a helpful assistant. You get the prior chat_history "
    "(as a list of messages) plus the user's follow-up question. "
    "Rewrite the follow-up so it can stand alone."
)

condense_question_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(CONDENSE_Q_SYSTEM),
    MessagesPlaceholder(variable_name="chat_history"),   # list is fine
    HumanMessagePromptTemplate.from_template("{question}")
])

# LLM and memory
import os, getpass
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

llm = ChatOpenAI(
    model_name="gpt-4o-2024-08-06",
    temperature=0.8,
    openai_api_key=os.environ["OPENAI_API_KEY"]
)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

Enter your OpenAI API key: ··········


<ipython-input-6-0a2615f7901a>:42: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(
<ipython-input-6-0a2615f7901a>:47: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [ ]:
# 4
import json, pathlib
from langchain.memory.chat_message_histories import ChatMessageHistory
from langchain.schema import HumanMessage, AIMessage, SystemMessage

MEM_FILE = f"{PAPER_DIR}/profS_memory.jsonl"

def load_memory():
    if not pathlib.Path(MEM_FILE).exists():
        return []
    with open(MEM_FILE, "r", encoding="utf-8") as f:
        return [json.loads(ln) for ln in f]

def append_memory(msgs):
    with open(MEM_FILE, "a", encoding="utf-8") as f:
        for m in msgs:
            f.write(json.dumps(m, ensure_ascii=False) + "\n")

# Load and wrap messages
long_term_history = load_memory()
print(f"Loaded {len(long_term_history)//2} previous Q&A pairs")

from langchain_core.messages import AIMessage, HumanMessage
from langchain.memory.chat_message_histories import ChatMessageHistory

chat_history = ChatMessageHistory()
for m in long_term_history:
    role = m["role"]
    content = m["content"]
    if role == "user":
        chat_history.add_user_message(content)
    elif role == "assistant":
        chat_history.add_ai_message(content)
    else:
        raise ValueError(f"Unknown role: {role}")

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    chat_memory=chat_history,
    output_key="answer",           # tell memory which field to store
)

from langchain.chains import ConversationalRetrievalChain

# build RAG chain
rag = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    get_chat_history=lambda hist: hist,          # keep list structure
    condense_question_prompt=condense_question_prompt,
    combine_docs_chain_kwargs={
        "prompt": prompt,
        "document_variable_name": "context",
    },
    return_source_documents=True,                # sources

    verbose=False,
)


Loaded 65 previous Q&A pairs


In [ ]:
# 5 · Chat loop
from langchain.schema import Document, SystemMessage, HumanMessage
from datetime import datetime
import textwrap

def wrap(txt: str) -> str:
    return textwrap.fill(txt, 90)

print("Ask Professor S anything :D  (type 'exit' to quit)")

try:
    while True:
        q = input("\nStudent: ").strip()
        if q.lower() in {"exit", "quit"}:
            break

        # 1 · RAG
        result  = rag.invoke({"question": q})
        answer  = result["answer"]
        # keep only chunks that really came from PDFs
        sources = [d for d in result.get("source_documents", [])
                   if d.metadata.get("type") == "paper"]

        # 2 · Decide if fallback is needed
        use_fallback = len(sources) == 0
        if use_fallback:
            print("(no relevant paper context → fallback to GPT-4o)")
            completion = llm.invoke([
                SystemMessage(
                    content="You are Professor S, a kind, patient, slightly quirky CS professor. "
                            "End every answer with ':D '"),
                HumanMessage(content=q)
            ])
            answer = completion.content.strip()

        # 3 · Display
        if not use_fallback:
            print("\nRetrieved context from papers:")
            for d in sources:
                print("•", d.page_content[:150].replace("\n", " "), "…")
        print("\nProfessor S:", wrap(answer))

        # 4 · Append to long-term memory file
        append_memory([
            {"role": "user",      "content": q},
            {"role": "assistant", "content": answer}
        ])

        # 5 · grow the vector store **only** with RAG answers
        if not use_fallback and sources:
            ts = datetime.utcnow().isoformat()
            vectordb.add_documents([
                Document(page_content=q,      metadata={"role": "user",      "ts": ts}),
                Document(page_content=answer, metadata={"role": "assistant", "ts": ts})
            ])

except KeyboardInterrupt:
    print("\nSession interrupted.")
finally:
    vectordb.save_local(INDEX_PATH)
    print("Index saved to", INDEX_PATH)


Ask Professor S anything :D  (type 'exit' to quit)

Student: Hi!
(no relevant paper context → fallback to GPT-4o)

Professor S: Hello there! How can I assist you today? :D

Student: Emm doppelganger?

Retrieved context from papers:
• ACM Reference Format: Siqi Guo, Minsoo Choi, Dominic Kao, and Christos Mousas. 2024. Collaborating with my Doppelgänger: The Effects of Self-similar A …
• Collaborating with my Doppelgänger: The Effects of Self-similar Appearance and Voice of a Virtual Character during a Jigsaw Puzzle Co-solving Task SIQ …

Professor S: Fantastic explanation! You've laid out the process of creating digital doppelgängers in
technology and virtual environments beautifully. Each step, from 3D scanning to AI
integration, plays a crucial role in developing realistic and interactive avatars. These
digital doubles have diverse applications, enhancing experiences in gaming, training, and
even social interactions in virtual spaces. If there are any specific areas you're curious
abo